In [5]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.regularizers import l2, l1
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.externals import joblib
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from time import time
from skimage import io
from skimage import data, segmentation, filters, color, img_as_float, img_as_ubyte, exposure, feature, measure, morphology
from skimage.color import rgb2gray
from skimage.morphology import square
from skimage.feature import hog

In [2]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [15]:
batch_size = 64
nb_classes = 10
nb_epoch = 1000

np.random.seed(1337) # for reproducibility

In [3]:
### Plan2 ###

In [10]:
features = joblib.load("./mldata/features_1200.mat")
labels = joblib.load("./mldata/lables_1200.mat")

features = np.array(features, 'int16')
labels = np.array(labels, 'int')

t0 = time()
def scale(X, eps = 0.001):
    # scale the data points s.t the columns of the feature space
    # (i.e the predictors) are within the range [0, 1]
    return (X - np.min(X, axis = 0)) / (np.max(X, axis = 0) + eps)

features = features.astype("float32")
features = scale(features)

print "escape time : ", round(time()-t0, 3), "s"

# scale the data to the range [0, 1] and then construct the training
# and testing splits
(X_train, X_test, y_train, y_test) = train_test_split(features, labels, test_size = 0.1)
print "the shape of training set %s rows, %s columns" %(trainX.shape[0], trainX.shape[1])
print "the shape of test set %s rows, %s columns" %(testX.shape[0], testX.shape[1])
print "the range of training set : %s ~ %s" %(trainX.min(),trainX.max())
print "the range of test set : %s ~ %s" %(testX.min(),testX.max())

escape time :  0.069 s
the shape of training set 5400 rows, 784 columns
the shape of test set 600 rows, 784 columns
the range of training set : 0.0 ~ 0.999996
the range of test set : 0.0 ~ 0.999996


In [11]:
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [16]:
model = Sequential()
model.add(Dense(784, 128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128, 128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128, 10))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms)

t0 = time()
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=2, validation_data=(X_test, y_test))
print "escape time : ", round(time()-t0, 3), "s"

score = model.evaluate(X_test, y_test, show_accuracy=True, verbose=0)
print 'Test score : %s' %score[0]
print 'Test accuracy : %s' %score[1]

Train on 5400 samples, validate on 600 samples
Epoch 0
0s - loss: 0.7444 - acc.: 0.8024 - val. loss: 0.3587 - val. acc.: 0.9151
Epoch 1
0s - loss: 0.3318 - acc.: 0.9235 - val. loss: 0.3355 - val. acc.: 0.9245
Epoch 2
0s - loss: 0.2925 - acc.: 0.9274 - val. loss: 0.2982 - val. acc.: 0.9318
Epoch 3
0s - loss: 0.2732 - acc.: 0.9294 - val. loss: 0.3163 - val. acc.: 0.9177
Epoch 4
0s - loss: 0.2546 - acc.: 0.9346 - val. loss: 0.2857 - val. acc.: 0.9318
Epoch 5
0s - loss: 0.2485 - acc.: 0.9326 - val. loss: 0.2790 - val. acc.: 0.9318
Epoch 6
0s - loss: 0.2391 - acc.: 0.9389 - val. loss: 0.2931 - val. acc.: 0.9224
Epoch 7
0s - loss: 0.2340 - acc.: 0.9398 - val. loss: 0.2834 - val. acc.: 0.9349
Epoch 8
0s - loss: 0.2239 - acc.: 0.9398 - val. loss: 0.2637 - val. acc.: 0.9333
Epoch 9
0s - loss: 0.2211 - acc.: 0.9404 - val. loss: 0.2902 - val. acc.: 0.9255
Epoch 10
0s - loss: 0.2154 - acc.: 0.9406 - val. loss: 0.3222 - val. acc.: 0.9365
Epoch 11
0s - loss: 0.2095 - acc.: 0.9444 - val. loss: 0.3428

In [17]:
model.